In [1]:
import math
import numpy as np
#import h5py
import matplotlib.pyplot as plt
#from matplotlib.pyplot import imread
#import scipy
#from PIL import Image
#i#ort pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.python.framework import ops
#from cnn_utils import *
from test_utils import summary, comparator

%matplotlib inline
np.random.seed(1)

In [2]:
cifar10 = tf.keras.datasets.cifar10
(X_train_orig, Y_train_orig), (X_test_orig, Y_test_orig) = cifar10.load_data()

In [3]:
Y_test_orig.shape

(10000, 1)

In [4]:
# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshape
Y_train = Y_train_orig
Y_test = Y_test_orig

In [ ]:
X_train = (X_train*2)-1
X_test =(X_test*2)-1


In [ ]:
X_train[0]

In [ ]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape)) #CONTIENE UN NUMERO CON LA CLASE DE LOS 5000 DE ENTRENAMIENTO
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

In [ ]:
index = 124
plt.imshow(X_train_orig[index]) #display sample training image
plt.show()

In [ ]:
# Obtener las clases únicas
classes = np.unique(Y_train)

# Imprimir las clases
print("Clases del conjunto de datos CIFAR-10:")
for cls in classes:
    print(cls)

In [ ]:
# Definir las clases correspondientes a cada número de etiqueta
class_names = ['avión', 'automóvil', 'pájaro', 'gato', 'ciervo',
               'perro', 'rana', 'caballo', 'barco', 'camión']

# Visualizar las clases asociadas con algunas imágenes
num_images = 5
fig, axes = plt.subplots(1, num_images, figsize=(15, 3))

for i in range(num_images):
    axes[i].imshow(X_train[i])
    axes[i].set_title(class_names[Y_train[i][0]])
    axes[i].axis('off')

plt.show()

In [ ]:
index = 3
plt.imshow(X_train[index]) #display sample training image
plt.show()

In [ ]:
# Definir las clases que nos interesan (gato y perro) PARA HACERLO SOLO CON SIGMOID
avion_class = 0  #AVION
rana_class = 6  # RANA

# Filtrar las imágenes y etiquetas para quedarnos solo con los gatos y perros
avion_indices = (Y_train == avion_class).squeeze()
rana_indices = (Y_train == rana_class).squeeze()
avion_rana_indices = avion_indices | rana_indices

X_train_avion_rana = X_train[avion_rana_indices]
Y_train_avion_rana = Y_train[avion_rana_indices]

# Verificar las dimensiones del nuevo conjunto de datos
print("Dimensiones de X_train_cat_dog:", X_train_avion_rana.shape)
print("Dimensiones de Y_train_cat_dog:", Y_train_avion_rana.shape)

In [ ]:
# Definir las clases que nos interesan (gato y perro) PARA HACERLO SOLO CON SIGMOID
avion_class = 0  #AVION
rana_class = 6  # RANA

# Filtrar las imágenes y etiquetas para quedarnos solo con los gatos y perros
avion_indices = (Y_test == avion_class).squeeze()
rana_indices = (Y_test == rana_class).squeeze()
avion_rana_indices = avion_indices | rana_indices

X_test_avion_rana = X_test[avion_rana_indices]
Y_test_avion_rana = Y_test[avion_rana_indices]

# Verificar las dimensiones del nuevo conjunto de datos
print("Dimensiones de X_train_cat_dog:", X_test_avion_rana.shape)
print("Dimensiones de Y_train_cat_dog:", Y_test_avion_rana.shape)

In [ ]:
Y_train_avion_rana

In [ ]:
Y_train_avion_rana= np.where(Y_train_avion_rana == rana_class, 0, 1) # 1 avion 0 rana
Y_test_avion_rana = np.where(Y_test_avion_rana == rana_class, 0, 1)

In [ ]:
Y_train_avion_rana

In [ ]:
index = 65
plt.imshow(X_train_avion_rana[index]) #display sample training image
plt.title(Y_train_avion_rana[index])
plt.show()
index = 0
plt.imshow(X_train_avion_rana[index]) #display sample training image
plt.title(Y_train_avion_rana[index])
plt.show()
index = 530
plt.imshow(X_train_avion_rana[index]) #display sample training image
plt.title(Y_train_avion_rana[index])
plt.show()
index = 753
plt.imshow(X_train_avion_rana[index]) #display sample training image
plt.title(Y_train_avion_rana[index])
plt.show()

In [ ]:
index = 65
plt.imshow(X_test_avion_rana[index]) #display sample training image
plt.title(Y_test_avion_rana[index])
plt.show()
index = 0
plt.imshow(X_test_avion_rana[index]) #display sample training image
plt.title(Y_test_avion_rana[index])
plt.show()
index = 530
plt.imshow(X_test_avion_rana[index]) #display sample training image
plt.title(Y_test_avion_rana[index])
plt.show()
index = 753
plt.imshow(X_test_avion_rana[index]) #display sample training image
plt.title(Y_test_avion_rana[index])
plt.show()

In [ ]:
# GRADED FUNCTION: happyModel

def ModelCifar10_S_Inicial():
    """
    Implements the forward propagation for the binary classification model:
    ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> FLATTEN -> DENSE
    
    Note that for simplicity and grading purposes, you'll hard-code all the values
    such as the stride and kernel (filter) sizes. 
    Normally, functions should take these values as function parameters.
    
    Arguments:
    None

    Returns:
    model -- TF Keras model (object containing the information for the entire training process) 
    """
    model = tf.keras.Sequential([

            tfl.ZeroPadding2D(padding=(0, 0),input_shape=(32,32,3)),
        

            #tfl.Conv2D(filters=2, kernel_size=7, strides=(5,5), input_shape=[32, 32, 3]), # de (32+2*0-5)/9+1=3+1=4 (4,4,2)
            tfl.Conv2D(filters=2, kernel_size=5, strides=(9,9), input_shape=[32, 32, 3]), # de (32+2*0-5)/9+1=3+1=4 (4,4,2)
        

            ## ReLU
            tfl.ReLU(max_value=None, negative_slope=0.0, threshold=0.0),
            ## Max Pooling 2D with default parameters
        
            tfl.MaxPool2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None), #(2,2,2)
            
            ## Flatten layer
        
            tfl.Flatten(), #de 8
            #tfl.Dense(5,activation="sigmoid"),
            #tfl.Dense(4,activation="sigmoid"),
            tfl.Dense(1,activation="sigmoid")

        ])
    
    return model

In [ ]:
def ModelCifar10_Inicial():
    # Establecer semillas aleatorias para NumPy
    tf.random.set_seed(64)
    # Define la entrada
    input_layer = tf.keras.Input(shape=(32, 32, 3))

    # Aplica el relleno, en este caso no se realiza padding
    padded_input = tfl.ZeroPadding2D(padding=(0, 0))(input_layer)

    # Aplica la convolución, 2 Filtros de tamaño 5x5x3 y un Stride de 9
    conv_output = tfl.Conv2D(filters=2, kernel_size=5, strides=(9, 9))(padded_input)

    # Aplica ReLU (mantiene valor si el valor es positivo, 0 en caso contrario)
    relu_output = tfl.ReLU(max_value=None, negative_slope=0.0, threshold=0.0)(conv_output)

    # Aplica MaxPooling 2x2
    pooling_output = tfl.MaxPool2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(relu_output)

    # Aplana la salida, proceso flatten
    flattened_output = tfl.Flatten()(pooling_output)

    # Capa completamente conectada, con función de activación sigmoid ('0' RANA,'1' AVION)
    dense_output = tfl.Dense(1, activation="sigmoid")(flattened_output)

    # Se crea modelo
    model = tf.keras.Model(inputs=input_layer, outputs=dense_output)

    return model



In [ ]:
cifar10_model_avion_rana_S = ModelCifar10_S_Inicial()
# Print a summary for each layer
#for layer in summary(cifar10_model_avion_rana_S):
#    print(layer)

cifar10_model_avion_rana = ModelCifar10_Inicial()
# Print a summary for each layer
#for layer in summary(cifar10_model_avion_rana):
#    print(layer)

In [ ]:
from tensorflow.keras.optimizers import Adam
#optimizer = Adam(learning_rate=0.0025) 
#optimizer = Adam(learning_rate=0.005)
optimizer = Adam(learning_rate=0.01) 
cifar10_model_avion_rana.compile(optimizer=optimizer,
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

In [ ]:
cifar10_model_avion_rana.summary()

In [ ]:
# Establecer semillas aleatorias para NumPy
#np.random.seed(64)

# Establecer semillas aleatorias para TensorFlow
#tf.random.set_seed(64)
cifar10_model_avion_rana.fit(X_train_avion_rana, Y_train_avion_rana, epochs=20, batch_size=512)

In [ ]:
cifar10_model_avion_rana.evaluate(X_test_avion_rana,Y_test_avion_rana)

In [ ]:
# Obtener los pesos y sesgos de cada capa
for layer in cifar10_model_avion_rana.layers:
    print("Capa:", layer.name)
    weights = layer.get_weights()
    if weights:
        for i, w in enumerate(weights):
            print("  Pesos {}:".format(i), w.shape)
    else:
        print("  No tiene pesos")

In [ ]:
for layer in cifar10_model_avion_rana.layers:
    print("Capa:", layer.name)
    weights = layer.get_weights()
    if weights:
        for i, w in enumerate(weights):
            print("  Pesos {}:".format(i))
            print(w)
    else:
        print("  No tiene pesos")

In [ ]:
conv_weights = cifar10_model_avion_rana.layers[2].get_weights()
conv_weights


In [ ]:
conv_weights = cifar10_model_avion_rana.layers[6].get_weights()
conv_weights